## Train: VOC 2007+2012, Test: VOC 2007 (IoU saves)

| **Loss**          | **IoU50**  | **IoU75**  | **IoU50:95** | **PIoU50** | **PIoU75** | **PIoU50:95** |
| ----------------  | ---------- | ---------- | ------------ | ---------- | ---------- | ------------- |
| ProbIoU           | **0.6470** | **0.3159** | **0.3368**   | 0.7029     | 0.5367     | **0.4819**    |
| GIoU              | 0.6389     | 0.2428     | 0.3033       | 0.7052     | 0.5332     | 0.4691        |
| DIoU              | 0.6357     | 0.2452     | 0.3015       | 0.7046     | 0.5337     | 0.4688        |
| CIoU              | 0.6467     | 0.2800     | 0.3225       | **0.7059** | **0.5372** | 0.4781        |
| IoU (default)     | 0.6340     | 0.2713     | 0.3110       | 0.6967     | 0.5301     | 0.4694        |

## Train: VOC 2007+2012, Test: VOC 2007 (ProbIoU saves)

| **Loss**          | **IoU50**  | **IoU75**  | **IoU50:95** | **PIoU50** | **PIoU75** | **PIoU50:95** |
| ----------------  | ---------- | ---------- | ------------ | ---------- | ---------- | ------------- |
| ProbIoU           | 0.6284     | 0.2288     | 0.2913       | 0.6971     | 0.5198     | 0.4611        |
| GIoU              | **0.6475** | **0.3070** | **0.3325**   | **0.7053** | **0.5408** | **0.4824**    |
| DIoU              | 0.6307     | 0.2298     | 0.2927       | 0.7048     | 0.5327     | 0.4657        |
| CIoU              | 0.6244     | 0.2045     | 0.2786       | 0.6984     | 0.5151     | 0.4538        |
| IoU (default)     | 0.6368     | 0.2826     | 0.3183       | 0.7007     | 0.5306     | 0.4743        |

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import keras.backend as K
from utils.eval import *
from generators.voc_generator import PascalVocGenerator
from utils.image import preprocess_image
import models

Using TensorFlow backend.


In [2]:
common_args = {
    'batch_size': 1,
    'image_min_side': 256,
    'image_max_side': 512,
    'preprocess_image': preprocess_image,
}
generator = PascalVocGenerator(
    '/datasets/dataset/VOCdevkit/VOC2007',
    'test',
    shuffle_groups=False,
    skip_truncated=False,
    skip_difficult=True,
    **common_args
)

In [3]:
model_path = './snapshots/pascal_resnet50_iou/finetuned/resnet50_pascal.h5'
# load retinanet model
K.clear_session()
K.set_learning_phase(1)
model = models.load_model(model_path, backbone_name='resnet50')
# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

W0423 02:47:45.750052 139737818224448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0423 02:47:45.750936 139737818224448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0423 02:47:45.762826 139737818224448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0423 02:47:45.772008 139737818224448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W042

In [4]:
average_precisions = evaluate_mAP(generator, model, method='iou')

# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations), generator.label_to_name(label),
          'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)
mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('IoU mAP0.5:0.95 = {:.4f}'.format(mean_ap))

Running network: 100% (4952 of 4952) |###| Elapsed Time: 0:03:48 Time:  0:03:48
Parsing annotations: 100% (4952 of 4952) || Elapsed Time: 0:00:01 Time:  0:00:01
Evaluating threshold 0.50: 100% (20 of 20) || Elapsed Time: 0:00:05 Time:  0:00:05
Evaluating threshold 0.55:   5% (1 of 20) || Elapsed Time: 0:00:00 ETA:  00:00:00

AP0.50 = 0.6368


Evaluating threshold 0.55: 100% (20 of 20) || Elapsed Time: 0:00:05 Time:  0:00:05
Evaluating threshold 0.60:   5% (1 of 20) || Elapsed Time: 0:00:00 ETA:  00:00:00

AP0.55 = 0.5972


Evaluating threshold 0.60: 100% (20 of 20) || Elapsed Time: 0:00:05 Time:  0:00:05
Evaluating threshold 0.65:   5% (1 of 20) || Elapsed Time: 0:00:00 ETA:  00:00:00

...

AP0.85 = 0.0743


Evaluating threshold 0.90: 100% (20 of 20) || Elapsed Time: 0:00:05 Time:  0:00:05
Evaluating threshold 0.95:   5% (1 of 20) || Elapsed Time: 0:00:00 ETA:  00:00:00

AP0.90 = 0.0165


Evaluating threshold 0.95: 100% (20 of 20) || Elapsed Time: 0:00:05 Time:  0:00:05


AP0.95 = 0.0009
num_fp=1513070.0, num_tp=56070.0
285 instances of class aeroplane with average precision: 0.3319
337 instances of class bicycle with average precision: 0.3513
459 instances of class bird with average precision: 0.3148
263 instances of class boat with average precision: 0.2107
469 instances of class bottle with average precision: 0.1477
213 instances of class bus with average precision: 0.4117
1201 instances of class car with average precision: 0.4202
358 instances of class cat with average precision: 0.4488
756 instances of class chair with average precision: 0.1672
244 instances of class cow with average precision: 0.3589
206 instances of class diningtable with average precision: 0.2547
489 instances of class dog with average precision: 0.4169
348 instances of class horse with average precision: 0.3607
325 instances of class motorbike with average precision: 0.3178
4528 instances of class person with average precision: 0.3035
480 instances of class pottedplant with ave

In [5]:
average_precisions = evaluate_mAP(generator, model, method='piou')

# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations), generator.label_to_name(label),
          'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)
mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('ProbIoU mAP0.5:0.95 = {:.4f}'.format(mean_ap))

Running network: 100% (4952 of 4952) |###| Elapsed Time: 0:03:01 Time:  0:03:01
Parsing annotations: 100% (4952 of 4952) || Elapsed Time: 0:00:01 Time:  0:00:01
Evaluating threshold 0.50: 100% (20 of 20) || Elapsed Time: 0:00:07 Time:  0:00:07
Evaluating threshold 0.55:   5% (1 of 20) || Elapsed Time: 0:00:00 ETA:   0:00:01

AP0.50 = 0.7007


Evaluating threshold 0.55: 100% (20 of 20) || Elapsed Time: 0:00:07 Time:  0:00:07
Evaluating threshold 0.60:   5% (1 of 20) || Elapsed Time: 0:00:00 ETA:  00:00:00

AP0.55 = 0.6864


Evaluating threshold 0.60: 100% (20 of 20) || Elapsed Time: 0:00:07 Time:  0:00:07
Evaluating threshold 0.65:   5% (1 of 20) || Elapsed Time: 0:00:00 ETA:   0:00:01

...

AP0.85 = 0.3174


Evaluating threshold 0.90: 100% (20 of 20) || Elapsed Time: 0:00:07 Time:  0:00:07
Evaluating threshold 0.95:   5% (1 of 20) || Elapsed Time: 0:00:00 ETA:  00:00:00

AP0.90 = 0.1492


Evaluating threshold 0.95: 100% (20 of 20) || Elapsed Time: 0:00:07 Time:  0:00:07


AP0.95 = 0.0155
num_fp=1489968.0, num_tp=79172.0
285 instances of class aeroplane with average precision: 0.5098
337 instances of class bicycle with average precision: 0.5341
459 instances of class bird with average precision: 0.4830
263 instances of class boat with average precision: 0.3506
469 instances of class bottle with average precision: 0.2574
213 instances of class bus with average precision: 0.5571
1201 instances of class car with average precision: 0.5870
358 instances of class cat with average precision: 0.6302
756 instances of class chair with average precision: 0.2770
244 instances of class cow with average precision: 0.4996
206 instances of class diningtable with average precision: 0.3814
489 instances of class dog with average precision: 0.6058
348 instances of class horse with average precision: 0.5460
325 instances of class motorbike with average precision: 0.5015
4528 instances of class person with average precision: 0.4764
480 instances of class pottedplant with ave